### License (MIT)
Copyright (c) 2016 Kirill Moskovtsev

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Model for Si with deformation potential $\Xi_u$ shear strain effects ($\Xi_{sh}$)
Run the 'Definitions' cell first, then run the cells for Si or Ge.
## Definitions:

In [1]:
'''Python 2.7'''
from sympy import *
import numpy as np
from itertools import combinations_with_replacement, permutations
from IPython.display import Math, display

def avg_wm(Xis, structure, eta_structure):
    res = sum_wetas_etam(Xis, structure, eta_structure)
    T_power = 0
    for i in structure:
        T_power = T_power + i
    return res/M**len(structure)/T**T_power

def sum_wetas_etam(Xis, structure, eta_structure):
    '''
    This is the most important function in the code.
    See explanation in a cell below.
    The version includes shear strain term, used when including mass effects.
    '''
    prod = 1
    M = len(Xis)
    for ns, ne in zip(structure, eta_structure):
        sumi = 0
        for i in range(M):
            #sumi = sumi + (einsum('ij,ij', Xis[i], eta_t) + Ds*eta_minus[i]**2)**ns*eta_minus[i]**ne
            sumi = sumi + (einsum('ij,ij', Xis[i], eta_t) + Ds*eta_minus[i]**2 + Ds4*eta_minus[i]**4)**ns*eta_minus[i]**ne
        prod = prod*sumi
    return expand(prod)


def einsum(string, *args):
    '''
    Analog of np.einsum which works for 'object' data,
    not only numbers
    Posted by Krastanov at http://stackoverflow.com/questions/15606937/how-do-i-get-numpy-einsum-to-play-well-with-sympy
    '''
    index_groups = map(list, string.split(','))
    assert len(index_groups) == len(args)
    tensor_indices_tuples = zip(index_groups, args)
    return reduce(einsum_for_two, tensor_indices_tuples)[1]

def einsum_for_two(tensor_indices1, tensor_indices2):
    '''
    component of einsum
    '''
    string1, tensor1 = tensor_indices1
    string2, tensor2 = tensor_indices2
    sum_over_indices = set(string1).intersection(set(string2))
    new_string = string1 + string2
    axes = ([], [])
    for i in sum_over_indices:
        new_string.remove(i)
        new_string.remove(i)
        axes[0].append(string1.index(i))
        axes[1].append(string2.index(i))
    return new_string, np.tensordot(tensor1, tensor2, axes)


def show_math(x, s = '', prnt = False):
    display(Math(s + latex(x)))
    if prnt:
        print s + latex(x)

def R(i,j):
    return Rational(i,j)
    
sd, su, N, T, F0, F1, F2, F3, F4, F32, J, Ds, Ds4= symbols("\\Xi_d \\Xi_u N T F_{1/2} {F_{1/2}}' {F_{1/2}}'' {F_{1/2}}'''" + \
                                                  " {F_{1/2}}'''' F_{3/2} J \\Xi_{s2} \\Xi_{s4}")
eta1, eta2, eta3, eta4, eta5, eta6, theta0, x, K = symbols("\\eta_1 \\eta_2 \\eta_3 \\eta_4 \\eta_5 \\eta_6 \\theta_0 x K")
eta_t = [[eta1, Rational(1,2)*eta6, Rational(1,2)*eta5],[Rational(1,2)*eta6, eta2, Rational(1,2)*eta4],\
         [Rational(1,2)*eta5, Rational(1,2)*eta4, eta3]]
eta_minus = [Rational(1,2)*eta4, Rational(1,2)*eta4, Rational(1,2)*eta5, \
             Rational(1,2)*eta5, Rational(1,2)*eta6, Rational(1,2)*eta6]
M = 6
unit = [[1,0,0], [0,1,0], [0,0,1]]
unitm = Matrix(unit)

Xis = []
for i in range(3):
    for j in [-1, 1]:
        a = np.reshape(np.array([0,0,0]), (1,3))
        a[0,i] = j
        at = np.reshape(a, (3,1))
        Xis.append(sd*unitm + su*np.dot(at, a))


**'sum_wetas_etam**(Xis, structure, eta_structure)':

Calculates the following expression:

$$
\left(\sum_\alpha \left(\delta E^{(\alpha)}\right)^{\text{structure[0]}}\eta_\alpha^{eta\_structure[0]}\right)...\left(\sum_\alpha \left(\delta E^{(\alpha)}\right)^{\text{structure[r]}}\eta_\alpha^{eta\_structure[r]}\right)
$$
where $\delta E^{(\alpha)}$ is the shift of valley $\alpha$, $\eta_\alpha$ defined as $\varepsilon_\alpha$ in the appendix of the paper, but $\eta_\alpha$ is written in Voigt notation.

If we ignore *'eta_structure'*, then $\Delta_\varepsilon$ structures from the paper are written as follows, for example:
$$
    \overline{\Delta_\varepsilon^k}\overline{\Delta_\varepsilon^n} = T^{-(k+n)}M^{-2}\left(\text{sum_wetas_etam}(Xis, (k,n), (0,0))\right)
$$

## Calculate expressions for $\delta c_{ij...}$ for doped Si:
$\Xi_{s2} \equiv \Xi_{sh}$;

$\Xi_{s4}$ is the hypothetical deformation potential for a quartic in strain shift of valleys, and it is assumed small.

In [7]:
#Free energy terms expressed through strain
Free1 = N*T*avg_wm(Xis, (1,), (0,))
Free2 = -N*T/F0*(Rational(1,2)*F1*(avg_wm(Xis, (2,), (0,)) - avg_wm(Xis, (1,1), (0,0)) ))

Free3 = N*T/F0*(Rational(1,6)*F2*(avg_wm(Xis, (3,),(0,)) - 3*avg_wm(Xis, (2,1),(0,0)) + \
                                  2*avg_wm(Xis, (1,1,1),(0,0,0))) )

Free4_1 = Rational(1,24)*N*T*F3/F0*(-6*avg_wm(Xis, (2,1,1), (0,0,0)) + 4*avg_wm(Xis, (3,1),(0,0)) - avg_wm(Xis, (4,),(0,)) +\
                                    3*avg_wm(Xis, (1,1,1,1),(0,0,0,0)))
Free4_2 = Rational(1,8)*N*T*F2**2/F0/F1*(avg_wm(Xis, (2,2),(0,0)) - 2*avg_wm(Xis, (2,1,1),(0,0,0)) + \
                                         avg_wm(Xis, (1,1,1,1), (0,0,0,0)) )

Free4 = expand(Free4_1 + Free4_2)

Free_all = expand(Free1 + Free2 + Free3 + Free4)
subs_dict = {eta1: eta1*x, eta2: eta2*x, eta3: eta3*x, eta4: eta4*x, eta5: eta5*x, eta6: eta6*x}
Free_all_x = Free_all.subs(subs_dict)
Free_xdict = collect(Free_all_x, x, evaluate=False)
Free2 = Free_xdict[x**2]
Free3 = Free_xdict[x**3]
Free4 = Free_xdict[x**4]

#Differentiate to find elastic constants:

dc11 = simplify(diff(Free2, eta1, eta1))
dc12 = simplify(diff(Free2, eta1, eta2))
dc44 = simplify(diff(Free2, eta4, eta4))
print "Second order:"
show_math(dc11, '\\delta c_{11} = ')
show_math(dc12, '\\delta c_{12} = ')
show_math(dc44, '\\delta c_{44} = ')


dc111 = diff(Free3, eta1, eta1, eta1)
dc112 = diff(Free3, eta1, eta1, eta2)
dc123 = diff(Free3, eta1, eta2, eta3)
dc144 = diff(Free3, eta1, eta4, eta4)
dc155 = diff(Free3, eta1, eta5, eta5)
dc456 = diff(Free3, eta4, eta5, eta6)

print "Third order:"
show_math(dc111, '\\delta c_{111} = ')
show_math(dc112, '\\delta c_{112} = ')
show_math(dc123, '\\delta c_{123} = ')
show_math(dc144, '\\delta c_{144} = ')
show_math(dc155, '\\delta c_{155} = ')
show_math(dc456, '\\delta c_{456} = ', prnt = False)



dc1111 = expand(diff(Free4, eta1, eta1, eta1, eta1))
dc1112 = expand(diff(Free4, eta1, eta1, eta1, eta2))
dc1122 = expand(diff(Free4, eta1, eta1, eta2, eta2))
dc1123 = expand(diff(Free4, eta1, eta1, eta2, eta3))
dc1144 = expand(diff(Free4, eta1, eta1, eta4, eta4))
dc1155 = expand(diff(Free4, eta1, eta1, eta5, eta5))
dc1244 = expand(diff(Free4, eta1, eta2, eta4, eta4))
dc1266 = expand(diff(Free4, eta1, eta2, eta6, eta6))
dc1456 = expand(diff(Free4, eta1, eta4, eta5, eta6))
dc4444 = expand(diff(Free4, eta4, eta4, eta4, eta4))
dc4455 = expand(diff(Free4, eta4, eta4, eta5, eta5))
print "Fourth order:"
show_math(dc1111, '\\delta c_{1111} = ')
show_math(dc1112, '\\delta c_{1112} = ')
show_math(dc1122, '\\delta c_{1122} = ')
show_math(dc1123, '\\delta c_{1123} = ')
show_math(dc1144, '\\delta c_{1144} = ')
show_math(dc1155, '\\delta c_{1155} = ')
show_math(dc1244, '\\delta c_{1244} = ')
show_math(dc1266, '\\delta c_{1266} = ')
show_math(dc1456, '\\delta c_{1456} = ')
show_math(dc4444, '\\delta c_{4444} = ', prnt = False)
show_math(dc4455, '\\delta c_{4455} = ', prnt = False)

Second order:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Third order:


...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Save single-crystal $c$-expressions
to reuse them in the 'mode_calculation.ipynb'

In [5]:
import pickle
c11, c12, c44, c111, c112, c123, c144, c155, c456 = \
symbols('c11 c12 c44 c111 c112 c123 c144 c155 c456')
c1111, c1112, c1122, c1123, c1144, c1155, c1244, c1266, c1456, c4444, c4455 = \
symbols('c1111 c1112 c1122 c1123 c1144 c1155 c1244 c1266 c1456 c4444 c4455')

c = {c11:dc11, c12:dc12, c44:dc44, c111:dc111, c112:dc112, c123:dc123, c144:dc144, \
            c155:dc155, c456:dc456, c1111:dc1111, c1112:dc1112, c1122:dc1122, c1123:dc1123, \
             c1144:dc1144,\
            c1155:dc1155, c1244:dc1244, c1266:dc1266, c1456:dc1456, c4444:dc4444, c4455:dc4455}
fname = './single_expressions/c_values_single.dat'
if not os.path.exists(os.path.dirname(fname)):
        os.makedirs(os.path.dirname(fname))
        print './single_expressions/ created'
ff = open(fname, 'w')
pickle.dump(c, ff)
ff.close()

# Germanium

In [2]:
def sum_wetas_etam(Xis, structure, eta_structure):
    prod = 1
    M = len(Xis)
    for ns, ne in zip(structure, eta_structure):
        sumi = 0
        for i in range(M):
            #sumi = sumi + (einsum('ij,ij', Xis[i], eta_t) + Ds*eta_minus[i]**2)**ns*eta_minus[i]**ne
            sumi = sumi + (einsum('ij,ij', Xis[i], eta_t))**ns
        prod = prod*sumi
    return expand(prod)

M = 4
unit_vectors = np.array([[1,1,1], [-1,1,1], [1,-1, 1], [-1, -1, 1]])/sqrt(3)
unitm = np.array([[1,0,0], [0,1,0], [0,0,1]])
#unitm = Matrix(unit)

Xis = []
for a in unit_vectors:
    #for j in [-1/sqrt(3), 1/sqrt(3)]:
    a = np.reshape(a, (1,3))
    #a[0,2] = j
    at = np.reshape(a, (3,1))
    Xis.append(sd*unitm + su*np.dot(at, a))


Free1 = N*T*avg_wm(Xis, (1,), (0,))
Free2 = -N*T/F0*(Rational(1,2)*F1*(avg_wm(Xis, (2,), (0,)) - avg_wm(Xis, (1,1), (0,0)) ))
#Free2 = simplify(Free2)



Free3 = N*T/F0*(Rational(1,6)*F2*(avg_wm(Xis, (3,),(0,)) - 3*avg_wm(Xis, (2,1),(0,0)) + \
                                  2*avg_wm(Xis, (1,1,1),(0,0,0))) )


Free4_1 = Rational(1,24)*N*T*F3/F0*(-6*avg_wm(Xis, (2,1,1), (0,0,0)) + 4*avg_wm(Xis, (3,1),(0,0)) - avg_wm(Xis, (4,),(0,)) +\
                                    3*avg_wm(Xis, (1,1,1,1),(0,0,0,0)))
Free4_2 = Rational(1,8)*N*T*F2**2/F0/F1*(avg_wm(Xis, (2,2),(0,0)) - 2*avg_wm(Xis, (2,1,1),(0,0,0)) + \
                                         avg_wm(Xis, (1,1,1,1), (0,0,0,0)) )

Free4 = expand(Free4_1 + Free4_2)

Free_all = expand(Free1 + Free2 + Free3 + Free4).subs({Ds: 0, Ds4: 0}) #no \Xi_sh for Ge
subs_dict = {eta1: eta1*x, eta2: eta2*x, eta3: eta3*x, eta4: eta4*x, eta5: eta5*x, eta6: eta6*x}
Free_all_x = Free_all.subs(subs_dict)
Free_xdict = collect(Free_all_x, x, evaluate=False)
Free2 = Free_xdict[x**2]
Free3 = Free_xdict[x**3]
Free4 = Free_xdict[x**4]

dc11 = simplify(diff(Free2, eta1, eta1))
dc12 = simplify(diff(Free2, eta1, eta2))
dc44 = simplify(diff(Free2, eta4, eta4))
print "Second order:"
show_math(dc11, '\\delta c_{11} = ')
show_math(dc12, '\\delta c_{12} = ')
show_math(dc44, '\\delta c_{44} = ')


dc111 = diff(Free3, eta1, eta1, eta1)
dc112 = diff(Free3, eta1, eta1, eta2)
dc123 = diff(Free3, eta1, eta2, eta3)
dc144 = diff(Free3, eta1, eta4, eta4)
dc155 = diff(Free3, eta1, eta5, eta5)
dc456 = diff(Free3, eta4, eta5, eta6)

print "Third order:"
show_math(dc111, '\\delta c_{111} = ')
show_math(dc112, '\\delta c_{112} = ')
show_math(dc123, '\\delta c_{123} = ')
show_math(dc144, '\\delta c_{144} = ')
show_math(dc155, '\\delta c_{155} = ')
show_math(dc456, '\\delta c_{456} = ', prnt = True)



dc1111 = expand(diff(Free4, eta1, eta1, eta1, eta1))
dc1112 = expand(diff(Free4, eta1, eta1, eta1, eta2))
dc1122 = expand(diff(Free4, eta1, eta1, eta2, eta2))
dc1123 = expand(diff(Free4, eta1, eta1, eta2, eta3))
dc1144 = expand(diff(Free4, eta1, eta1, eta4, eta4))
dc1155 = expand(diff(Free4, eta1, eta1, eta5, eta5))
dc1244 = expand(diff(Free4, eta1, eta2, eta4, eta4))
dc1266 = expand(diff(Free4, eta1, eta2, eta6, eta6))
dc1456 = expand(diff(Free4, eta1, eta4, eta5, eta6))
dc4444 = expand(diff(Free4, eta4, eta4, eta4, eta4))
dc4455 = expand(diff(Free4, eta4, eta4, eta5, eta5))
print "Fourth order:"
show_math(dc1111, '\\delta c_{1111} = ')
show_math(dc1112, '\\delta c_{1112} = ')
show_math(dc1122, '\\delta c_{1122} = ')
show_math(dc1123, '\\delta c_{1123} = ')
show_math(dc1144, '\\delta c_{1144} = ')
show_math(dc1155, '\\delta c_{1155} = ')
show_math(dc1244, '\\delta c_{1244} = ')
show_math(dc1266, '\\delta c_{1266} = ')
show_math(dc1456, '\\delta c_{1456} = ')
show_math(dc4444, '\\delta c_{4444} = ', prnt = True)
show_math(dc4455, '\\delta c_{4455} = ', prnt = True)

Second order:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Third order:


...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

\delta c_{4444} = - \frac{N \Xi_u^{4} {F_{1/2}}'''}{81 F_{1/2} T^{3}} + \frac{N \Xi_u^{4} {F_{1/2}}''^{2}}{27 F_{1/2} T^{3} {F_{1/2}}'}


<IPython.core.display.Math object>

\delta c_{4455} = - \frac{N \Xi_u^{4} {F_{1/2}}'''}{81 F_{1/2} T^{3}} + \frac{N \Xi_u^{4} {F_{1/2}}''^{2}}{81 F_{1/2} T^{3} {F_{1/2}}'}


In [3]:
print '\delta c_{{11}} &= {}'.format(latex(dc11)) + '\\\\'
print '\delta c_{{12}} & = {}'.format(latex(dc12)) + '\\\\'
print '\delta c_{{44}} & = {}'.format(latex(dc44)) + '\\\\'
print '\delta c_{{111}} & = {}'.format(latex(dc111)) + '\\\\'
print '\delta c_{{112}} & = {}'.format(latex(dc112)) + '\\\\'
print '\delta c_{{123}} & = {}'.format(latex(dc123)) + '\\\\'
print '\delta c_{{144}} & = {}'.format(latex(dc144)) + '\\\\'
print '\delta c_{{155}} & = {}'.format(latex(dc155)) + '\\\\'
print '\delta c_{{456}} & = {}'.format(latex(dc456)) + '\\\\'
print '\delta c_{{1111}} & = {}'.format(latex(dc1111)) + '\\\\'
print '\delta c_{{1112}} & = {}'.format(latex(dc1112)) + '\\\\'
print '\delta c_{{1122}} & = {}'.format(latex(dc1122)) + '\\\\'
print '\delta c_{{1123}} & = {}'.format(latex(dc1123)) + '\\\\'
print '\delta c_{{1144}} & = {}'.format(latex(dc1144)) + '\\\\'
print '\delta c_{{1155}} & = {}'.format(latex(dc1155)) + '\\\\'
print '\delta c_{{1244}} & = {}'.format(latex(dc1244)) + '\\\\'
print '\delta c_{{1266}} & = {}'.format(latex(dc1266)) + '\\\\'
print '\delta c_{{1456}} & = {}'.format(latex(dc1456)) + '\\\\'
print '\delta c_{{4444}} & = {}'.format(latex(dc4444)) + '\\\\'
print '\delta c_{{4455}} & = {}'.format(latex(dc4455))

\delta c_{11} &= 0\\
\delta c_{12} & = 0\\
\delta c_{44} & = - \frac{N \Xi_u^{2} {F_{1/2}}'}{9 F_{1/2} T}\\
\delta c_{111} & = 0\\
\delta c_{112} & = 0\\
\delta c_{123} & = 0\\
\delta c_{144} & = 0\\
\delta c_{155} & = 0\\
\delta c_{456} & = \frac{N \Xi_u^{3} {F_{1/2}}''}{27 F_{1/2} T^{2}}\\
\delta c_{1111} & = 0\\
\delta c_{1112} & = 0\\
\delta c_{1122} & = 0\\
\delta c_{1123} & = 0\\
\delta c_{1144} & = 0\\
\delta c_{1155} & = 0\\
\delta c_{1244} & = 0\\
\delta c_{1266} & = 0\\
\delta c_{1456} & = 0\\
\delta c_{4444} & = - \frac{N \Xi_u^{4} {F_{1/2}}'''}{81 F_{1/2} T^{3}} + \frac{N \Xi_u^{4} {F_{1/2}}''^{2}}{27 F_{1/2} T^{3} {F_{1/2}}'}\\
\delta c_{4455} & = - \frac{N \Xi_u^{4} {F_{1/2}}'''}{81 F_{1/2} T^{3}} + \frac{N \Xi_u^{4} {F_{1/2}}''^{2}}{81 F_{1/2} T^{3} {F_{1/2}}'}
